In [11]:
# LOAD SAVED MODEL & MAKE PREDICTIONS 

import pickle
import json
import numpy as np
import pandas as pd
from pathlib import Path
from tensorflow.keras.models import load_model
import xgboost as xgb

print("="*80)
print("LOADING NVDA PREDICTOR MODEL")
print("="*80)

model_path = Path('nvda_models/nvda_predictor_nextday_v1')

def engineer_features(data):
    """Engineer all features for prediction"""
    df = data.copy()
    
    # Returns and volatility
    df['Returns'] = df['Close'].pct_change()
    df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
    df['Volatility_20'] = df['Returns'].rolling(20).std()
    df['Volatility_60'] = df['Returns'].rolling(60).std()
    
    # Volume ratios
    df['Volume_Ratio'] = df['Volume'] / df['Volume'].rolling(20).mean()
    df['Dollar_Volume'] = df['Close'] * df['Volume']
    
    # Price spreads
    df['HL_Spread'] = (df['High'] - df['Low']) / df['Close']
    df['CO_Spread'] = (df['Close'] - df['Open']) / df['Open']
    
    # Rate of change
    df['ROC_10'] = (df['Close'] - df['Close'].shift(10)) / df['Close'].shift(10)
    df['ROC_30'] = (df['Close'] - df['Close'].shift(30)) / df['Close'].shift(30)
    
    # Support/Resistance
    df['Resistance'] = df['High'].rolling(20).max()
    df['Support'] = df['Low'].rolling(20).min()
    df['SR_Position'] = (df['Close'] - df['Support']) / (df['Resistance'] - df['Support'])
    
    # Trade sizes and trends
    df['Avg_Trade_Size'] = df['Dollar_Volume'] / df['Volume']
    
    # Handle missing SMA values
    if 'SMA5' not in df.columns:
        df['SMA5'] = df['Close'].rolling(5).mean()
    if 'SMA50' not in df.columns:
        df['SMA50'] = df['Close'].rolling(50).mean()
    if 'RSI' not in df.columns:
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
    if 'MACD' not in df.columns:
        ema12 = df['Close'].ewm(span=12).mean()
        ema26 = df['Close'].ewm(span=26).mean()
        df['MACD'] = ema12 - ema26
    if 'SMA_20' not in df.columns:
        df['SMA_20'] = df['Close'].rolling(20).mean()
    if 'EMA_50' not in df.columns:
        df['EMA_50'] = df['Close'].ewm(span=50).mean()
    if 'BB_upper' not in df.columns:
        bb_middle = df['Close'].rolling(20).mean()
        bb_std = df['Close'].rolling(20).std()
        df['BB_upper'] = bb_middle + 2 * bb_std
        df['BB_lower'] = bb_middle - 2 * bb_std
    
    df['Trend_Strength'] = abs(df['SMA5'] - df['SMA50']) / df['SMA50']
    df['Volatility_Regime'] = df['Volatility_20'] / df['Volatility_60']
    df['Price_Change'] = df['Close'].pct_change()
    
    return df.fillna(0)

try:
    # Load LSTM model
    print("\n📥 Loading models...")
    lstm_model = load_model(str(model_path / 'lstm_gru_model.h5'))
    print("✓ LSTM-GRU model loaded")
    
    # Load LSTM scaler
    with open(model_path / 'lstm_scaler.pkl', 'rb') as f:
        lstm_scaler = pickle.load(f)
    print("✓ LSTM scaler loaded")
    
    # Load XGBoost model
    xgb_model = xgb.XGBRegressor()
    xgb_model.load_model(str(model_path / 'xgboost_model.json'))
    print("✓ XGBoost model loaded")
    
    # Load ARIMA-GARCH data
    with open(model_path / 'arima_garch_data.pkl', 'rb') as f:
        arima_garch_data = pickle.load(f)
    print("✓ ARIMA-GARCH data loaded")
    
    # Load metadata
    with open(model_path / 'lstm_metadata.json', 'r') as f:
        lstm_metadata = json.load(f)
    print("✓ LSTM metadata loaded")
    
    with open(model_path / 'xgboost_metadata.pkl', 'rb') as f:
        xgb_metadata = pickle.load(f)
    print("✓ XGBoost metadata loaded")
    
    # Load config
    with open(model_path / 'config.json', 'r') as f:
        config = json.load(f)
    print("✓ Configuration loaded")
    
    # Load last predictions
    with open(model_path / 'predictions.json', 'r') as f:
        last_predictions = json.load(f)
    print("✓ Last predictions loaded")
    
    # Load confidence intervals
    with open(model_path / 'confidence_intervals.json', 'r') as f:
        confidence_intervals = json.load(f)
    print("✓ Confidence intervals loaded")
    
    # Load risk metrics
    with open(model_path / 'risk_metrics.json', 'r') as f:
        risk_metrics = json.load(f)
    print("✓ Risk metrics loaded")
    
    print("\n✅ All models loaded successfully!")
    print(f"Model saved on: {config['saved_date']}")
    print(f"Last data date: {config['last_data_date']}")
    
    # ========== MAKE NEW PREDICTIONS ==========
    print("\n" + "="*80)
    print("MAKING NEW PREDICTIONS")
    print("="*80)
    
    # Load and engineer features
    print("\n🔧 Engineering features...")
    df = pd.read_csv('NVDA_dataset_updated.csv')
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df = df.sort_index()
    
    df = engineer_features(df)
    
    print(f"Latest data date: {df.index[-1].strftime('%Y-%m-%d')}")
    
    current_price = df['Close'].iloc[-1]
    print(f"Current Price: ${current_price:.2f}")
    
    # 1. LSTM-GRU prediction
    print("\n📊 Model Predictions:")
    seq_len = lstm_metadata['sequence_length']
    feature_cols = lstm_metadata['feature_cols']
    close_idx = lstm_metadata['close_idx']
    
    last_sequence = df[feature_cols].iloc[-seq_len:].fillna(0).values
    last_sequence_scaled = lstm_scaler.transform(last_sequence)
    last_sequence_scaled = last_sequence_scaled.reshape(1, seq_len, -1)
    
    lstm_pred_scaled = lstm_model.predict(last_sequence_scaled, verbose=0)[0, 0]
    close_min = lstm_scaler.data_min_[close_idx]
    close_range = lstm_scaler.data_range_[close_idx]
    lstm_pred = float(lstm_pred_scaled * close_range + close_min)
    
    print(f"  LSTM-GRU: ${lstm_pred:.2f} ({(lstm_pred/current_price - 1)*100:+.2f}%)")
    
    # 2. XGBoost prediction
    xgb_features = xgb_metadata['features']
    last_features = df[xgb_features].iloc[-1].fillna(0).values.reshape(1, -1)
    xgb_change = float(xgb_model.predict(last_features)[0])
    xgb_pred = current_price * (1 + xgb_change)
    
    print(f"  XGBoost: ${xgb_pred:.2f} ({(xgb_pred/current_price - 1)*100:+.2f}%)")
    
    # 3. ARIMA-GARCH prediction - FIXED: Convert to float
    arima_forecast = arima_garch_data['forecast_mean']
    if isinstance(arima_forecast, np.ndarray):
        arima_forecast = float(arima_forecast[0])
    else:
        arima_forecast = float(arima_forecast)
    
    arima_pred = current_price * np.exp(arima_forecast)
    print(f"  ARIMA-GARCH: ${arima_pred:.2f} ({(arima_pred/current_price - 1)*100:+.2f}%)")
    
    # 4. Monte Carlo median (from saved data)
    mc_median = confidence_intervals['median']
    print(f"  Monte Carlo: ${mc_median:.2f} ({(mc_median/current_price - 1)*100:+.2f}%)")
    
    # 5. Ensemble prediction using saved weights
    weights = config['ensemble_weights']
    ensemble = (
        weights['ARIMA-GARCH'] * arima_pred +
        weights['XGBoost'] * xgb_pred +
        weights['LSTM-GRU'] * lstm_pred +
        weights['Monte_Carlo'] * mc_median
    )
    
    print(f"\n  🎯 ENSEMBLE: ${ensemble:.2f} ({(ensemble/current_price - 1)*100:+.2f}%)")
    
    # 6. Compare with last saved prediction
    print(f"\nLast Saved Prediction:")
    print(f"  Date: {config['last_data_date']}")
    print(f"  Price: ${last_predictions['current_price']:.2f}")
    print(f"  Target: ${last_predictions['ensemble']:.2f}")
    
    # ========== CONFIDENCE INTERVALS ==========
    print("\n" + "="*80)
    print("CONFIDENCE INTERVALS (from saved model)")
    print("="*80)
    print(f"95% CI: [${confidence_intervals['lower_95']:.2f}, ${confidence_intervals['upper_95']:.2f}]")
    print(f"75% CI: [${confidence_intervals['lower_75']:.2f}, ${confidence_intervals['upper_75']:.2f}]")
    print(f"Median: ${confidence_intervals['median']:.2f}")
    
    # ========== RISK METRICS ==========
    print("\n" + "="*80)
    print("RISK METRICS (from saved model)")
    print("="*80)
    print(f"Expected Return: {risk_metrics['expected_return']:.2f}%")
    print(f"VaR (95%): {risk_metrics['VaR_95']:.2f}%")
    print(f"CVaR (95%): {risk_metrics['CVaR_95']:.2f}%")
    print(f"Sharpe Ratio: {risk_metrics['sharpe_ratio']:.2f}")
    print(f"Volatility: {risk_metrics['volatility_forecast']:.2f}%")
    
    # ========== TRADING SIGNAL ==========
    print("\n" + "="*80)
    print("TRADING SIGNAL")
    print("="*80)
    
    change = (ensemble / current_price - 1) * 100
    
    if change > 2:
        signal = "🟢 STRONG BUY"
        action = "Consider aggressive entry"
    elif change > 0.5:
        signal = "🟢 BUY"
        action = "Consider moderate entry"
    elif change < -2:
        signal = "🔴 STRONG SELL"
        action = "Consider exit or short"
    elif change < -0.5:
        signal = "🔴 SELL"
        action = "Consider reducing position"
    else:
        signal = "🟡 HOLD"
        action = "Wait for better setup"
    
    print(f"\nSignal: {signal}")
    print(f"Action: {action}")
    print(f"Target: ${ensemble:.2f}")
    print(f"Expected Change: {change:+.2f}%")
    print(f"Stop Loss: ${current_price * (1 + risk_metrics['VaR_95']/100):.2f}")
    
    # ========== SAVE RESULTS ==========
    results_df = pd.DataFrame({
        'Timestamp': [pd.Timestamp.now()],
        'Data_Date': [df.index[-1].strftime('%Y-%m-%d')],
        'Current_Price': [current_price],
        'LSTM_Pred': [lstm_pred],
        'XGBoost_Pred': [xgb_pred],
        'ARIMA_Pred': [arima_pred],
        'MC_Median': [mc_median],
        'Ensemble': [ensemble],
        'Change_%': [change],
        'Signal': [signal],
        'Lower_95CI': [confidence_intervals['lower_95']],
        'Upper_95CI': [confidence_intervals['upper_95']],
        'Sharpe_Ratio': [risk_metrics['sharpe_ratio']]
    })
    
    # Append to history
    history_file = 'prediction_history.csv'
    if Path(history_file).exists():
        history_df = pd.read_csv(history_file)
        results_df = pd.concat([history_df, results_df], ignore_index=True)
    
    results_df.to_csv(history_file, index=False)
    print(f"\n✅ Results saved to: {history_file}")
    
    # Save latest prediction
    latest_pred = {
        'timestamp': pd.Timestamp.now().isoformat(),
        'data_date': df.index[-1].strftime('%Y-%m-%d'),
        'current_price': float(current_price),
        'predictions': {
            'lstm_gru': float(lstm_pred),
            'xgboost': float(xgb_pred),
            'arima_garch': float(arima_pred),
            'monte_carlo': float(mc_median),
            'ensemble': float(ensemble)
        },
        'change_pct': float(change),
        'signal': signal,
        'confidence_intervals': confidence_intervals,
        'risk_metrics': risk_metrics
    }
    
    with open('latest_prediction.json', 'w') as f:
        json.dump(latest_pred, f, indent=2)
    print(f"✅ Latest prediction saved to: latest_prediction.json")
    
    print("\n" + "="*80)
    print("PREDICTION COMPLETE!")
    print("="*80)
    
except FileNotFoundError as e:
    print(f"\n❌ Model files not found. Please check the path: {model_path}")
    print(f"Error: {str(e)}")
except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    import traceback
    traceback.print_exc()

LOADING NVDA PREDICTOR MODEL

📥 Loading models...
✓ LSTM-GRU model loaded
✓ LSTM scaler loaded
✓ XGBoost model loaded
✓ ARIMA-GARCH data loaded
✓ LSTM metadata loaded
✓ XGBoost metadata loaded
✓ Configuration loaded
✓ Last predictions loaded
✓ Confidence intervals loaded
✓ Risk metrics loaded

✅ All models loaded successfully!
Model saved on: 2025-10-12T22:57:31.694146
Last data date: 2025-10-10

MAKING NEW PREDICTIONS

🔧 Engineering features...
Latest data date: 2025-10-10
Current Price: $183.16

📊 Model Predictions:
  LSTM-GRU: $191.04 (+4.30%)
  XGBoost: $182.54 (-0.34%)
  ARIMA-GARCH: $183.28 (+0.07%)
  Monte Carlo: $183.05 (-0.06%)

  🎯 ENSEMBLE: $186.58 (+1.87%)

Last Saved Prediction:
  Date: 2025-10-10
  Price: $183.16
  Target: $186.58

CONFIDENCE INTERVALS (from saved model)
95% CI: [$182.07, $184.05]
75% CI: [$182.65, $183.46]
Median: $183.05

RISK METRICS (from saved model)
Expected Return: 1.87%
VaR (95%): -0.60%
CVaR (95%): -0.73%
Sharpe Ratio: 6.79
Volatility: 4.36%

TRA